In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from array import *
from pickle import dump

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
np.random.seed(123)

import pyarrow as pa
import pyarrow.parquet as pq

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
consumo_parquet = pq.read_table('drive/MyDrive/K-Project/datasources/THC-consumption-data/THC-consumption_date_count-parquet/part.0.parquet')
consumo = consumo_parquet.to_pandas()
consumo

,id,datetime,value
__null_dask_index__,,,
0,13,2019-01-01,96
1,13,2019-01-02,96
2,13,2019-01-03,96
3,13,2019-01-04,96
4,13,2019-01-05,96
...,...,...,...
59823338,162329,2020-11-06,9
59823339,162341,2020-11-06,57
59823340,162355,2020-11-06,81


# Customers

In [ ]:
customers_path = "drive/MyDrive/K-Project/datasources/customers.csv"
df_customers = pd.read_csv(customers_path, low_memory=False)

In [ ]:
df_customers

,customer_id,tension,tariff,power,department,section,segment,zone
0,2,BT 230 V,TCB,3700,3,5,4,19
1,3,BT 230 V,TRS,2200,3,4,14,7
2,5,BT 230 V,TRS,2200,13,3,303,19
3,6,BT 230 V,TCB,3700,19,1,124,800
4,9,BT 230 V,TCB,3700,15,8,302,600


# Appliances

In [ ]:
appliances_path = "drive/MyDrive/K-Project/datasources/appliances.csv"
df_appliances = pd.read_csv(appliances_path, low_memory=False)

In [ ]:
df_appliances

,customer_id,meter_id,appl_desc,appl_type
0,170000,00124B0002CBAF2D,Desumificador (Tercer dormitorio),dehumidifier
1,170000,00124B0018D7080C,HOME_CONSUMPTION (Sala de estar),site meter
2,170001,00124B0002CBAC43,Microndas (Cocina),microwave
3,170001,00124B0002CBAD5E,Cocina eléctrica (Cocina),electric oven
4,170001,00124B0002CBAE00,Lavarropa (Baño),washing machine
5,170001,00124B0002CBAEBD,Heladera (Cocina),fridge
6,170001,00124B0002CBAF74,Caloventilador,electric air heater
7,170001,00124B0002CC9051,Termofón (Baño),electric water heating appliance
8,170001,00124B0018D6F755,HOME_CONSUMPTION (Salon),site meter
9,170004,00124B0002CBABF1,Heladera (Cocina),fridge


# Appliance consumption

In [ ]:
appl_cons_path = "drive/MyDrive/K-Project/datasources/appliance_consumption_data.csv"
df_appl_cons = pd.read_csv(appl_cons_path, low_memory=False)

In [ ]:
df_appl_cons

,datetime,meter_id,aenergy,apower,apower_ph1,apower_ph2,apower_ph3
0,1566943205,00124B0002CBABF1,2.0,120.0,120.0,0.0,0.0
1,1566943205,00124B0002CBAC13,0.0,0.0,0.0,0.0,0.0
2,1566943205,00124B0002CBAC43,0.0,0.0,0.0,0.0,0.0
3,1566943205,00124B0002CBAC6D,0.0,1.0,1.0,0.0,0.0
4,1566943205,00124B0002CBACAF,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
592683,1568653507,00124B0002CBABF1,2.0,113.0,113.0,0.0,0.0
592684,1568653507,00124B0002CBAC6D,4.0,1.0,1.0,0.0,0.0
592685,1568653507,00124B0002CBACAF,18.0,1075.0,1075.0,0.0,0.0
592686,1568653507,00124B0002CBAE49,0.0,0.0,0.0,0.0,0.0


# Disaggregated Energy Consumption by appliance subset total consumption data

In [ ]:
dec_path = "drive/MyDrive/K-Project/datasources/DEC-total_consumption_data.csv"
df_dec = pd.read_csv(dec_path, low_memory=False)

In [ ]:
print(df_dec)

          datetime          meter_id  aenergy  aenergy_ph1  aenergy_ph2  \
0       1566943205  00124B0018D6F607      4.0      19830.0          0.0   
1       1566943205  00124B0018D6F752      4.0      29250.0          0.0   
2       1566943205  00124B0018D6F755      8.0    1845536.0          0.0   
3       1566943205  00124B0018D6FB1E      3.0      12563.0          0.0   
4       1566943206  00124B0018D7080C      0.0          0.0          0.0   
...            ...               ...      ...          ...          ...   
144251  1568653505  00124B0018D6F74C      2.0     390499.0        405.0   
144252  1568653505  00124B0018D6F755      6.0    2137274.0          0.0   
144253  1568653505  00124B0018D6FB1E      1.0      15612.0          0.0   
144254  1568653505  00124B0018D7080C      0.0          0.0          0.0   
144255  1568653507  00124B0018D6F752     33.0       5073.0          0.0   

        aenergy_ph3  renergy  renergy_ph1  renergy_ph2  renergy_ph3  ...  \
0               0.0    

# Electric Water Heater consumption subset consumption data timers

### Consumption data timers

In [ ]:
ewh_timers_path = "drive/MyDrive/K-Project/datasources/EWH-consumption_data_timers.csv"
df_ewh_timers = pd.read_csv(ewh_timers_path, low_memory=False)

In [ ]:
df_ewh_timers

,datetime,id,power,voltage
0,1562102961,866131033548393,0,223
1,1562103021,866131033548393,0,226
2,1562103081,866131033548393,0,226
3,1562103141,866131033548393,0,226
4,1562103201,866131033548393,0,226
...,...,...,...,...
29041579,1574236585,868814048615657,0,227
29041580,1574236598,868814048615756,0,234
29041581,1574236601,868814048615921,0,35
29041582,1574236561,868814048616242,0,230


### Consumption data customers

In [ ]:
ewh_customers_path = "drive/MyDrive/K-Project/datasources/EWH-consumption_data_timers.csv"
df_ewh_customers = pd.read_csv(ewh_customers_path, low_memory=False)

In [ ]:
df_ewh_customers

,datetime,id,power,voltage
0,1562102961,866131033548393,0,223
1,1562103021,866131033548393,0,226
2,1562103081,866131033548393,0,226
3,1562103141,866131033548393,0,226
4,1562103201,866131033548393,0,226
...,...,...,...,...
29041579,1574236585,868814048615657,0,227
29041580,1574236598,868814048615756,0,234
29041581,1574236601,868814048615921,0,35
29041582,1574236561,868814048616242,0,230


# Total Household Consumption subset

### Consumption 15 min mean

In [ ]:
thc_15min_path = "drive/MyDrive/K-Project/datasources/THC-consumption-data/consumption_15min_mean.csv"
df_thc_15min = pd.read_csv(thc_15min_path, low_memory=False)

In [ ]:
df_thc_15min

,datetime,datetime.1,value
0,0,0,0.096
1,0,15,0.091
2,0,30,0.087
3,0,45,0.083
4,1,0,0.079
...,...,...,...
91,22,45,0.111
92,23,0,0.109
93,23,15,0.107
94,23,30,0.104


### Consumption date count

In [ ]:
thc_date_count_path = "drive/MyDrive/K-Project/datasources/THC-consumption-data/consumption_date_count.csv"
df_thc_date_count = pd.read_csv(thc_date_count_path, low_memory=False)

In [ ]:
df_thc_date_count

,id,datetime,value
0,13,2019-01-01,96
1,13,2019-01-02,96
2,13,2019-01-03,96
3,13,2019-01-04,96
4,13,2019-01-05,96
...,...,...,...
59823338,162329,2020-11-06,9
59823339,162341,2020-11-06,57
59823340,162355,2020-11-06,81
59823341,162365,2020-11-06,9


### Consumption weekday mean

In [ ]:
thc_weekday_path = "drive/MyDrive/K-Project/datasources/THC-consumption-data/consumption_weekday_mean.csv"
df_thc_weekday = pd.read_csv(thc_weekday_path, low_memory=False)

In [ ]:
df_thc_weekday

,datetime,value
0,0,0.087
1,1,0.088
2,2,0.088
3,3,0.088
4,4,0.088
5,5,0.087
6,6,0.086
